In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import FunctionTransformer


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

In [3]:
DANE=r'Dane\ames2000_NAfix.csv'
dane=pd.read_csv(DANE)
dane.head(15)

,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Street,Alley,Lot.Shape,Land.Contour,Utilities,Lot.Config,...,Pool.Area,Pool.QC,Fence,Misc.Feature,Misc.Val,Mo.Sold,Yr.Sold,Sale.Type,Sale.Condition,SalePrice
0,120,RL,34,3901,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,8,2007,New,Partial,204000
1,20,RL,70,8400,Pave,None,Reg,Lvl,AllPub,Corner,...,0,None,MnPrv,None,0,4,2010,WD,Normal,126000
2,85,RL,60,7200,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,MnPrv,None,0,4,2009,WD,Normal,133900
3,90,RL,64,7018,Pave,None,Reg,Bnk,AllPub,Inside,...,0,None,None,None,0,6,2009,WD,Alloca,118858
4,60,RL,111,16259,Pave,None,Reg,Lvl,AllPub,Corner,...,0,None,None,None,0,9,2006,New,Partial,342643
5,20,RL,50,4280,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,None,None,0,9,2009,WD,Normal,88750
6,20,RL,155,20064,Pave,None,IR1,Low,AllPub,Inside,...,0,None,None,None,0,5,2007,WD,Normal,279000
7,20,RM,60,7200,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,8,2006,COD,Normal,105000
8,20,RL,70,9100,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,9,2006,WD,Normal,235000
9,70,C (all),None,6449,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,None,None,0,3,2010,WD,Abnorml,93369


In [4]:
dane.dtypes # sprawdzamy typy kolumn


MS.SubClass        int64
MS.Zoning         object
Lot.Frontage      object
Lot.Area           int64
Street            object
                   ...  
Mo.Sold            int64
Yr.Sold            int64
Sale.Type         object
Sale.Condition    object
SalePrice          int64
Length: 80, dtype: object

In [5]:
column_int = []
column_obj = []
i=0
for typ in dane.dtypes:
    if dane.dtypes[i]=='object':
        column_int.append(dane)
    
    elif dane.dtypes[i]=='int64':
        column_obj.append(dane)
    i+=1


In [6]:
print(len(column_int)) # sprawdzam ile jest kolumn danego rodzaju
print(len(column_obj))

54
26


In [7]:
#nastepnie sprawdzam ilośc null w danych
number=dane.select_dtypes([np.number])
number.count()
non_number = dane.select_dtypes([np.object])
non_number.count()

MS.Zoning         2000
Lot.Frontage      2000
Street            2000
Alley             2000
Lot.Shape         2000
Land.Contour      2000
Utilities         2000
Lot.Config        2000
Land.Slope        2000
Neighborhood      2000
Condition.1       2000
Condition.2       2000
Bldg.Type         2000
House.Style       2000
Roof.Style        2000
Roof.Matl         2000
Exterior.1st      2000
Exterior.2nd      2000
Mas.Vnr.Type      1984
Mas.Vnr.Area      2000
Exter.Qual        2000
Exter.Cond        2000
Foundation        2000
Bsmt.Qual         1999
Bsmt.Cond         1999
Bsmt.Exposure     1997
BsmtFin.Type.1    1999
BsmtFin.SF.1      2000
BsmtFin.Type.2    1998
BsmtFin.SF.2      2000
Bsmt.Unf.SF       2000
Total.Bsmt.SF     2000
Heating           2000
Heating.QC        2000
Central.Air       2000
Electrical        2000
Bsmt.Full.Bath    2000
Bsmt.Half.Bath    2000
Kitchen.Qual      2000
Functional        2000
Fireplace.Qu      2000
Garage.Type       2000
Garage.Yr.Blt     2000
Garage.Fini

In [8]:
missings = len(dane) - dane.count().sort_values(ascending=True)
missings

Mas.Vnr.Type      16
Bsmt.Exposure      3
Garage.Finish      2
BsmtFin.Type.2     2
Bsmt.Cond          1
                  ..
Exterior.1st       0
Roof.Matl          0
Roof.Style         0
Year.Built         0
SalePrice          0
Length: 80, dtype: int64

In [9]:
dane_copy =dane.copy()
dane_copy.head(5)

,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Street,Alley,Lot.Shape,Land.Contour,Utilities,Lot.Config,...,Pool.Area,Pool.QC,Fence,Misc.Feature,Misc.Val,Mo.Sold,Yr.Sold,Sale.Type,Sale.Condition,SalePrice
0,120,RL,34,3901,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,8,2007,New,Partial,204000
1,20,RL,70,8400,Pave,None,Reg,Lvl,AllPub,Corner,...,0,None,MnPrv,None,0,4,2010,WD,Normal,126000
2,85,RL,60,7200,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,MnPrv,None,0,4,2009,WD,Normal,133900
3,90,RL,64,7018,Pave,None,Reg,Bnk,AllPub,Inside,...,0,None,None,None,0,6,2009,WD,Alloca,118858
4,60,RL,111,16259,Pave,None,Reg,Lvl,AllPub,Corner,...,0,None,None,None,0,9,2006,New,Partial,342643


In [10]:
dane_copy= dane.copy()
dane_copy.head(2)

,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Street,Alley,Lot.Shape,Land.Contour,Utilities,Lot.Config,...,Pool.Area,Pool.QC,Fence,Misc.Feature,Misc.Val,Mo.Sold,Yr.Sold,Sale.Type,Sale.Condition,SalePrice
0,120,RL,34,3901,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,8,2007,New,Partial,204000
1,20,RL,70,8400,Pave,None,Reg,Lvl,AllPub,Corner,...,0,None,MnPrv,None,0,4,2010,WD,Normal,126000


In [11]:
X = dane_copy.drop(["SalePrice"], axis=1)
y = dane_copy["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [12]:
# Inspired from stackoverflow.com/questions/25239958
class MissingImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = 'missing'
        return self
    def transform(self, X, y=None):
        return X.fillna('missing')

In [13]:
'''class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]'''

'class DataFrameSelector(BaseEstimator, TransformerMixin):\n    def __init__(self, attribute_names):\n        self.attribute_names = attribute_names\n    def fit(self, X, y=None):\n        return self\n    def transform(self, X):\n        return X[self.attribute_names]'

In [14]:
def select_cat_type(X):
    return X.select_dtypes(include='object')

def select_num_type(X):
    return X.select_dtypes(include=np.number)

In [16]:
cat_pipeline = Pipeline([
        ("select_cat", FunctionTransformer(select_cat_type)),
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ("cat_encoder", OneHotEncoder(sparse=False,handle_unknown='ignore')),
        ('standardscaler', StandardScaler())
    ])

cat_pipeline.fit(X_train)

Pipeline(steps=[('select_cat',
                 FunctionTransformer(func=<function select_cat_type at 0x0000025FE49E85E0>)),
                ('imputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('cat_encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('standardscaler', StandardScaler())])

In [17]:
num_pipeline = Pipeline([
        ("select_numeric", FunctionTransformer(select_num_type)),
        ("imputer", SimpleImputer(strategy="median")),
        ('standardscaler', StandardScaler())
    ])
num_pipeline.fit(X_train)

Pipeline(steps=[('select_numeric',
                 FunctionTransformer(func=<function select_num_type at 0x0000025FE49E8670>)),
                ('imputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [18]:
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])
preprocess_pipeline

FeatureUnion(transformer_list=[('num_pipeline',
                                Pipeline(steps=[('select_numeric',
                                                 FunctionTransformer(func=<function select_num_type at 0x0000025FE49E8670>)),
                                                ('imputer',
                                                 SimpleImputer(strategy='median')),
                                                ('standardscaler',
                                                 StandardScaler())])),
                               ('cat_pipeline',
                                Pipeline(steps=[('select_cat',
                                                 FunctionTransformer(func=<function select_cat_type at 0x0000025FE49E85E0>)),
                                                ('imputer',
                                                 SimpleImputer(fill_value='missing',
                                                               strategy='constant')),
         

In [19]:
seed=123
kfold = StratifiedKFold(n_splits=5, random_state=seed)

c:\users\barto\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [20]:
from sklearn.linear_model import LinearRegression

pipe_0 = Pipeline([('preprocessing', preprocess_pipeline), ('classifier', LinearRegression(n_jobs=-1))])

reg = pipe_0.fit(X_train, y_train)

reg.score(X_test, y_test)



0.7859010538841564

In [ ]:
from sklearn.linear_model import LogisticRegression

pipe_1 = Pipeline([('preprocessing', preprocess_pipeline), ('classifier', LogisticRegression(C=10**10,n_jobs=-1,max_iter=10000,random_state=42))])

#param_grid_1 = {
           # 'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100,1000]
#}

#grid_1 = GridSearchCV(pipe_1, param_grid_1, cv=kfold, return_train_score=True)

pipe_1.fit(X_train, y_train,)






In [21]:
pipe_1.score(X_train, y_train)

NameError: name 'pipe_1' is not defined

In [ ]:
'''from sklearn.metrics import accuracy_score
grid_1.best_params_
print("model score: %.3f" % grid_1.score(X_test, y_test))
accuracy_score(model.predict(X_test), y_test)'''

In [22]:
from sklearn.tree import DecisionTreeRegressor

pipe_2 = Pipeline([('preprocessing', preprocess_pipeline), 
                   ('classifier', DecisionTreeRegressor())])

param_grid_2 = {
            'classifier__max_depth': [5,9,10,11,20,30],
            'classifier__min_samples_split': [30,40,50,100,200,300],
            'classifier__max_leaf_nodes': [3,4,10,14,15,16,20,30,40,100,200,300]
}


grid_2 = GridSearchCV(pipe_2, param_grid_2, cv=kfold, return_train_score=True,refit= True)

grid_2.fit(X_train, y_train)
grid_2.best_params_

c:\users\barto\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


KeyboardInterrupt: 

In [ ]:
grid_2.score(X_train, y_train)

In [ ]:
from sklearn import  metrics


models = []

models.append(('LR', grid_1.best_estimator_))
#models.append(('DecisionTreeClassifier', grid_2.best_estimator_))
#models.append(('RandomForestClassifier', grid_3.best_estimator_))


precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
roc_auc_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test , model.predict(X_test)) ))
    print("recall_score: {}".format( metrics.recall_score(y_test , model.predict(X_test)) ))
    print("f1_score: {}".format( metrics.f1_score(y_test , model.predict(X_test)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_test , model.predict(X_test)) ))
    
    print("roc_auc_score: {}".format( metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]) ))
    
    precision_score.append(metrics.precision_score(y_test , model.predict(X_test)))
    recall_score.append(metrics.recall_score(y_test , model.predict(X_test)))
    f1_score.append( metrics.f1_score(y_test , model.predict(X_test)))
    accuracy_score.append(metrics.accuracy_score(y_test , model.predict(X_test)))
              
    roc_auc_score.append(metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]))
    

In [ ]:
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score,
     'roc_auc_score' : roc_auc_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=['SVM linear','SVM rbf','LR','KNN','DecisionTreeClassifier','BaggingClassifier','RandomForestClassifier','ExtraTreesClassifier', 'AdaBoostClassifier','GradientBoostingClassifier','XGBClassifier','voting','XGBClassifier r'])
df